# **Apigee GenAI Workshop**

<!-- <table align="left">
    <td style="text-align: center">
      <a href="https://colab.research.google.com/github/GoogleCloudPlatform/apigee-samples/blob/genai-workshop/genai-workshop.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
      </a>
    </td>
    <td style="text-align: center">
      <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fapigee-samples%2Fgenai-workshop%2Fgenai-workshop.ipynb">
        <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
      </a>
    </td>    
    <td style="text-align: center">
      <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/apigee-samples/genai-workshop/genai-workshop.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
      </a>
    </td>
    <td style="text-align: center">
      <a href="https://github.com/GoogleCloudPlatform/apigee-samples/blob/genai-workshop/genai-workshop.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
      </a>
    </td>
</table> -->

## Introduction

Welcome to Google's Apigee GenAI Workshop! 

This hands-on workshop will equip you with the knowledge and skills to leverage the power of Generative AI within your API ecosystem. Through practical exercises and real-world examples, you'll learn how to seamlessly integrate Large Language Models (LLMs) with Apigee, Google's leading API management platform. Get ready to unlock new possibilities and explore the exciting world of GenAI and APIs!

You should already have a lab instance up and running with all the necessary artifacts (Apigee, Vertex AI, Vertex DB, etc) provisioned for you to use. 

First, lets install the necessary dependencies to run the labs

## Install dependencies

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install langchain_google_vertexai
!pip install langchain-openai
!pip install google-cloud-aiplatform
!pip install google-cloud-tasks
!pip install openai

## Initialize notebook variables

You can fetch all the variables from your lab instance

* **PROJECT_ID**: The default GCP project provisioned
* **LOCATION**: The default GCP Region where the project is provisioned.
* **APIGEE_HOSTNAME**:  The hostname of the Apigee instance provisioned

In [11]:
# Define project information
PROJECT_ID = ""  # @param {type:"string"}
LOCATION = ""  # @param {type:"string"}
APIGEE_HOSTNAME = "" # @param {type:"string"}

---

## Lab: LLM-Logging with Apigee

Logging both prompts and candidate responses from large language models (LLMs) allows for detailed analysis and improvement of the model's performance over time. By examining past interactions, AI practitioners can identify patterns leading to refinements in the training data or model architecture. Furthermore, by examining the prompts security teams can detect malicious intent, such as attempts to extract sensitive information or generate harmful content.

Additionally, logging the generated candidates provides insights into the LLM's behavior and helps identify any biases or vulnerabilities in the model itself. This information can then be used to improve security measures, fine-tune the model, and mitigate potential risks associated with LLM usage.

![image](https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-logging/images/llm-logging.png?raw=1)


### Benefits of Logging with Apigee and Google Cloud Logging

* **Seamless logging**: Effortlessly capture prompts, candidate responses, and metadata without complex coding.
* **Scalable and secure**: Leverage Google Cloud's infrastructure for reliable and secure log management.

### How does it work?

1. Prompt request is receved by an Apigee Proxy.
2. Apigee extracts prompt and candidate responses.
3. Apigee logs prompt and candidate responses to Cloud Logging.

### Test Sample

Apigee allows you to seamlessly send logs to Cloud Logging using native integration with the [Message Logging](https://cloud.google.com/apigee/docs/api-platform/reference/policies/message-logging-policy#cloudloggingelement) policy. This sample also includes a message chunking solution that allows logging very long messages (ex. 1M tokens supported by Gemini) and connecting them together using a unique message identifier.

With the following cell you'll be able to invoke an LLM and both prompt and candidate resposne will be logged in Cloud Logging.

In [ ]:
from langchain_google_vertexai import VertexAI

if not PROJECT_ID or PROJECT_ID == "":
    raise ValueError("Please set your PROJECT_ID")
if not LOCATION or LOCATION == "":
    raise ValueError("Please set your LOCATION")
if not APIGEE_HOSTNAME or APIGEE_HOSTNAME == "":
    raise ValueError("Please set your APIGEE_HOSTNAME")

API_ENDPOINT = "https://"+APIGEE_HOSTNAME+"/v1/samples/llm-logging"

# Initialize Langchain
model = VertexAI(
      project=PROJECT_ID,
      location=LOCATION,
      api_endpoint=API_ENDPOINT,
      api_transport="rest",
      streaming=True,
      model_name="gemini-1.5-pro")

prompts = ["Provide an explanation about the benefits of using sunscreen. Make sure to make it as long as a novel."]

for prompt in prompts:
  print(model.invoke(prompt))

### Explore and Analyze Logs with Cloud Logging

1. Navigate to the Cloud Logging Explorer by [**clicking here**](https://console.cloud.google.com/logs/query?_ga=2.194228271.307340908.1727018794-898542846.1726863667).

2. Set the query filter. Make sure to replace the `PROJECT_ID` with the Apigee project ID:

  ```
  logName="projects/PROJECT_ID/logs/apigee"
  ```
3. Run the query and explore the logs. See example below:

  ![image](https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-logging/images/logs-explorer.png?raw=1)


Congratulations! You've successfully deployed the llm-logging proxy and tested the ability to log the request and responses from subsequent LLM's.

---

## Lab: LLM-Routing with Apigee

- This is a sample Apigee proxy to demonstrate the routing capabilities of Apigee across different LLM providers. In this sample we will use Google VertexAI, Mistral and HuggingFace as the LLM providers
- The framework will easily help onboarding other providers using configurations

![architecture](https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-routing/images/arch.jpg?raw=1)

### Pre-requisites

This lab requires you to have a HuggingFace Access Token and Mistral AI API Key. The provisioned lab instance **does not** include both.


To create a HuggingFace Access Token:
- You wll need to create an account in [HuggingFace](https://huggingface.co)
- Go to settings, click `Access Tokens` and then click the "Create new token" button
- Choose `Read` for token type, provide a name and then hit "Create token"
- Copy the token below

In [ ]:
HUGGINGFACE_TOKEN = ""  # @param {type:"string"}

To create a Mistral AI API Key:
- You wll need to create an account in [HuggingFace](https://mistral.ai )
- Click `API Keys` and then click the "Create new key" button
- Provide a name and then hit "Create key"
- Copy the key below

In [ ]:
MISTRALAI_KEY="" # @param {type:"string"}

### Benefits of Routing with Apigee:

* **Configuration Driven Routing**: All the routing logic are driven through configuration which makes onboarding very easy
* **Security**: Irrespective of the model and providers, Apigee will secure the endpoints
* **Consistency**: Apigee can offer that layer of consistency to work with any LLM SDKs that are being used

### Update HuggingFace and Mistral AI credentials in Apigee KVM Store

We need to update the HuggingFace and Mistral AI credentials in the Apigee KVM store

In [ ]:
import json
import requests

access_token = !gcloud auth print-access-token

url = 'https://apigee.googleapis.com/v1/organizations/'+PROJECT_ID+'/environments/eval/keyvaluemaps/llm-routing-v1-modelprovider-config/entries'
headers = {'Authorization': 'Bearer '+access_token[0], 'Content-type': 'application/json'}

entry = 'huggingface__token'
resp = requests.put(url+"/"+entry, headers = headers, data=json.dumps({"name": entry,"value": HUGGINGFACE_TOKEN}))
if resp.status_code == 200:
    print("HuggingFace Access Token updated successfully")
else:
    print (resp.text)

entry = 'mistral__token'
resp = requests.put(url+"/"+entry, headers = headers, data=json.dumps({"name": entry,"value": MISTRALAI_KEY}))
if resp.status_code == 200:
    print("Mistral AI API Key updated successfully")
else:
    print (resp.text)

### Test Sample

We will need an API Key that is already provisioned in your lab instance. Go to Apigee in your GCP console. In the left hand menu, select `Apps` to see the list of Apps. Click the `llm-routing-app` app and copy the `Key` from the `Credentials` section

In [ ]:
ROUTING_SAMPLE_APIKEY="" # @param {type:"string"}

In [ ]:
if not PROJECT_ID or PROJECT_ID == "":
    raise ValueError("Please set your PROJECT_ID")
if not LOCATION or LOCATION == "":
    raise ValueError("Please set your LOCATION")
if not APIGEE_HOSTNAME or APIGEE_HOSTNAME == "":
    raise ValueError("Please set your APIGEE_HOSTNAME")
if not ROUTING_SAMPLE_APIKEY or ROUTING_SAMPLE_APIKEY == "":
    raise ValueError("Please set your ROUTING_SAMPLE_APIKEY")

API_ENDPOINT = "https://"+APIGEE_HOSTNAME+"/v1/samples/llm-routing/"

PROMPT="Suggest name for a flower shop"

### Select an LLM Provider

Select a provider from the dropdown. This will automatically set the model name used by the SDKs

Try picking different providers from the dropdown above. You will see that the same SDK is able to call the Apigee endpoint serving responses from different providers

In [ ]:
import sys
from google.colab import auth

llm_provider = "select" # @param ["select","google", "huggingface", "mistral"]

if llm_provider == "google":
    model = "google/gemini-1.5-flash"
elif llm_provider == "mistral":
    model = "open-mistral-nemo"
elif llm_provider == "huggingface":
    model = "meta-llama/Llama-3.2-11B-Vision-Instruct"
else:
    raise ValueError("Invalid LLM provider")

### Using OpenAI SDK

In [ ]:
import openai

openai.api_key = ROUTING_SAMPLE_APIKEY
openai.base_url = API_ENDPOINT
openai.default_headers = {"x-apikey": ROUTING_SAMPLE_APIKEY, "x-llm-provider": llm_provider}

completion = openai.chat.completions.create(
    model=model,
    messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": PROMPT
        }
      ]
    }
  ]
)
print(f"Using the OpenAI SDK, fetching the response from \"{model}\" provided by \"{llm_provider}\"")
print("\n")
print(completion.choices[0].message.content)

### Using Langchain SDK

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=model,
    api_key=ROUTING_SAMPLE_APIKEY,
    base_url=API_ENDPOINT,
    default_headers = {"x-apikey": ROUTING_SAMPLE_APIKEY, "x-llm-provider": llm_provider}
)
messages = [
  {
    "role": "user",
    "content": [
      {
        "type": "text",
        "text": PROMPT
      }
    ]
  }
]
print(f"Using the Langchain SDK, fetching the response from \"{model}\" provided by \"{llm_provider}\"")
print("\n")
print(llm.invoke(messages).content)

Congratulations! You've successfully deployed the routing proxy and tested the ability to route calls to different LLM providers.

---

## Lab: LLM-Token-Limits with Apigee